In [63]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pylab import rcParams
random_state = 42
import autoreload
%load_ext autoreload
%autoreload 2


import pickle

from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,recall_score,precision_score
from treeinterpreter import treeinterpreter as ti
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier,ExtraTreeClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier,BaggingClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [64]:
df = pd.read_csv("./data/final.csv")

In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3904 entries, 0 to 3903
Data columns (total 33 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   FSSLTDE2  3904 non-null   float64
 1   FSSTDDE2  3904 non-null   float64
 2   FSERNDED  3904 non-null   float64
 3   RAWNET    3904 non-null   float64
 4   FSASSET   3904 non-null   float64
 5   TANF_IND  3904 non-null   float64
 6   RAWERND   3904 non-null   float64
 7   REALPROP  3904 non-null   float64
 8   FSOTHUN   3904 non-null   float64
 9   FSNONCIT  3904 non-null   float64
 10  CERTHHSZ  3904 non-null   float64
 11  FSSLTDED  3904 non-null   float64
 12  FSTANF    3904 non-null   float64
 13  FSTOTDED  3904 non-null   float64
 14  FSNK0T4   3904 non-null   float64
 15  FSCSEXP   3904 non-null   float64
 16  FSNETINC  3904 non-null   float64
 17  FSWAGES   3904 non-null   float64
 18  FSERNDE2  3904 non-null   float64
 19  TPOV      3904 non-null   float64
 20  FSNELDER  3904 non-null   floa

In [66]:
X = df.drop(columns = ['CAT_ELIG'])
y = df['CAT_ELIG']

In [67]:
#baseline, NULL Model
y.value_counts(normalize=True)

CAT_ELIG
1.0    0.625512
2.0    0.374488
Name: proportion, dtype: float64

In [68]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.3,random_state=random_state)

In [69]:
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

In [70]:
np.savetxt('./data/TrainTest/X_train.csv',X_train,delimiter=',')
np.savetxt('./data/TrainTest/y_train.csv',y_train,delimiter=',')
np.savetxt('./data/TrainTest/X_test.csv',X_test,delimiter=',')
np.savetxt('./data/TrainTest/y_test.csv',y_test,delimiter=',')

In [71]:
pca = PCA(n_components=10,random_state=42)
pca.fit(X_train)
X_train_pc = pca.transform(X_train)
X_test_pc = pca.transform(X_test)

In [72]:
models = {
    'LogReg': LogisticRegression(),
    'Decision Tree':DecisionTreeClassifier(),
    'Random Forest':RandomForestClassifier(),
    'Gradient Boost':GradientBoostingClassifier(),
    'Ada Boost':AdaBoostClassifier(),
    'SVC':SVC(),
    'Naive Bayes':GaussianNB()}

In [73]:
def final(fullname, dataset):
    """Returns a sliced dataset of columns found in corr_features list."""
    fullname_df = pd.DataFrame()
    for feature in fullname:
        try:
            fullname_df[feature] = dataset[feature]
        except:
            continue
    fullname_df['CAT_ELIG']=dataset['CAT_ELIG']
    return fullname_df

In [74]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import recall_score, precision_score

In [75]:
final = pd.DataFrame(columns=['name', 'cross_val_train', 'cross_val_test', 'test_recall', 'test_precision'])


In [76]:
from sklearn.linear_model import LogisticRegression

# Increase the number of iterations
model = LogisticRegression(max_iter=200)

In [ ]:
for name, model in models.items():
    results = {}
    results['name'] = name
    model.fit(X_train, y_train)
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)

    results['cross_val_train'] = np.mean(cross_val_score(model, X_train, y_train, cv=4))
    results['cross_val_test'] = np.mean(cross_val_score(model, X_test, y_test, cv=4))
    results['test_recall'] = recall_score(y_test, y_pred_test)
    results['test_precision'] = precision_score(y_test, y_pred_test)

    # Convert results to a DataFrame and concatenate
    results_df = pd.DataFrame([results])
    final = pd.concat([final, results_df], ignore_index=True)

print(final)

C:\Users\pmanc\AppData\Local\Programs\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\pmanc\AppData\Local\Programs\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative

In [ ]:
final.set_index('name')

In [ ]:
for name,model in models.items():
    name = model.fit(X_train,y_train)
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    print(name)
    print(classification_report(y_test, y_pred_test))
    print('\n')

In [ ]:
rf = RandomForestClassifier()
params={'max_depth':[None,3,4],
       'max_features':[None,'auto'],
       'n_estimators':[75,100,125]}
rf_gs = GridSearchCV(rf,param_grid=params)
rf_gs.fit(X_train,y_train)
print(rf_gs.best_score_)
rf_gs.best_params_

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train,y_train)

In [ ]:
instances = X_test[[735]]
instances

In [ ]:
ft_list = []
prediction, bias, contributions = ti.predict(rf, instances)
print( "Prediction", prediction)
print( "Bias (trainset prior)", bias)
print ("Feature contributions:")
for c, feature in zip(contributions[0], 
                             X.columns):
    ft_list.append((feature, np.round(c, 2)))
    print (feature, c)
    
labels, values = zip(*ft_list)

In [ ]:
df1 = pd.DataFrame(ft_list,columns=['feature','array'])
df2 = pd.DataFrame(df1["array"].to_list(), columns=['pred_0', 'pred_1'])
coef_df = pd.concat([df1,df2],axis=1).drop(columns=['array'])
coef_df.to_csv('./data/2018_indicators/coef.csv',index=None)

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
plt.title('Random Forest range of Coefficients Effect on SNAP \n (the larger the range, the more impact on predicting SNAP)')
plt.grid(zorder=0,alpha = 0.2)
xs = np.arange(len(labels))
ax.bar(xs,coef_df['pred_0'], label = 'pred 0')
ax.bar(xs,coef_df['pred_1'],label = 'pred 1')
ax.axhline(y=0, linestyle='--', color='black', linewidth=4)
ax.set_xticks(coef_df.index)
ax.set_xticklabels(coef_df['feature'],rotation = 45)
plt.legend()
plt.savefig('./images/rf_corr.png');

In [ ]:
et = ExtraTreeClassifier()
params={'max_depth':[None,3,4],
       'max_features':[None,'auto'],
       'max_leaf_nodes':[5,10]}
et_gs = GridSearchCV(et,param_grid=params)
et_gs.fit(X_train,y_train)
print(et_gs.best_score_)
et_gs.best_params_

In [ ]:
bag = BaggingClassifier()
bag.fit(X_train,y_train)
y_pred_train = bag.predict(X_train)
y_pred_test = bag.predict(X_test)
print(f'cross_val_train = {np.mean(cross_val_score(model,X_train_pc,y_train,cv=4))}')
print(f'cross_val_test = {np.mean(cross_val_score(model,X_test_pc,y_test,cv=4))}')
print(f'test_recall = {recall_score(y_test, y_pred_test)}')
print(f'test_precision = {precision_score(y_test, y_pred_test)}')

In [ ]:
0.9509512425417362
{'bag__n_estimators': 10,
 'gb__max_depth': 10,
 'gb__subsample': 0.8,
 'rf__bootstrap': False,
 'rf__max_depth': None,
 'rf__n_estimators': 1000}

In [ ]:
vote = VotingClassifier([
    ('rf',RandomForestClassifier(bootstrap=False,n_estimators=1000)),
    ('gb',GradientBoostingClassifier(max_depth=10,subsample=0.8)),
    ('bag',BaggingClassifier(n_estimators = 10))
])

In [ ]:
vote.fit(X_train,y_train)

In [ ]:
filename = 'final_model.sav'
pickle.dump(vote, open(filename, 'wb'))